In [ ]:
# script to add training reactions to a training reactions reaction.py and dictionary.txt

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import rmgpy.data.thermo
import rmgpy.data.kinetics
import rmgpy.reaction
import rmgpy.species
import rmgpy

import rmgpy.chemkin

In [2]:
def duplicate_exists(test_entry):
    for entry in training_depo.entries:
        if training_depo.entries[entry].item.is_isomorphic(test_entry.item):
            return True
    return False

In [3]:
# load the new training reactions
new_training_rxns = 'new_kinetics'  # path to training reactions
ark_kinetics_database = rmgpy.data.kinetics.KineticsDatabase()
ark_kinetics_database.load_libraries(new_training_rxns)
print(f'{len(ark_kinetics_database.libraries[""].entries)} new reactions loaded')

48 new reactions loaded


In [4]:
# Load Kinetics Database (quick, automatically loads recommended families)
family = 'Disproportionation'
ref_library_path = os.path.join(rmgpy.settings['database.directory'], 'kinetics')
ref_database = rmgpy.data.kinetics.KineticsDatabase()
ref_database.load(
    ref_library_path,
    libraries=[],
    families=[family]
)
training_depo = ref_database.families[family].get_training_depository()
print(len(training_depo.entries), 'training reactions')

193 training reactions


In [5]:
species_dict = training_depo.get_species(os.path.join(rmgpy.settings['database.directory'], 'kinetics', 'families', family, 'training', 'dictionary.txt'))

In [6]:
def get_species(sp):
    # look for the species in the training depo and return it if it already exists
    for key in species_dict.keys():
        if sp.is_isomorphic(species_dict[key]):
            return species_dict[key]

In [ ]:
# ark_kinetics_database.libraries[""].entries[entry].item.reactants[0].label

In [7]:
for entry in training_depo.entries:
    print(training_depo.entries[entry].item.reactants[0].molecule[0].get_all_labeled_atoms())

{'*1': <Atom 'C.'>}
{'*2': <Atom 'C'>, '*3': <Atom 'C.'>, '*4': <Atom 'H'>}
{'*2': <Atom 'C'>, '*3': <Atom 'C.'>, '*4': <Atom 'H'>}
{'*1': <Atom 'C..'>}
{'*1': <Atom 'H.'>}
{'*1': <Atom 'C.'>}
{'*2': <Atom 'C'>, '*3': <Atom 'C.'>, '*4': <Atom 'H'>}
{'*1': <Atom 'C.'>}
{'*1': <Atom 'C.'>}
{'*2': <Atom 'C'>, '*3': <Atom 'C.'>, '*4': <Atom 'H'>}
{'*2': <Atom 'C'>, '*3': <Atom 'C.'>, '*4': <Atom 'H'>}
{'*1': <Atom 'C.'>}
{'*1': <Atom 'C.'>}
{'*1': <Atom 'O.'>}
{'*1': <Atom 'H.'>}
{'*2': <Atom 'C'>, '*3': <Atom 'C.'>, '*4': <Atom 'H'>}
{'*1': <Atom 'C..'>}
{'*1': <Atom 'S.'>}
{'*1': <Atom 'H.'>}
{'*1': <Atom 'C.'>}
{'*1': <Atom 'C.'>}
{'*2': <Atom 'C'>, '*3': <Atom 'C.'>, '*4': <Atom 'H'>}
{'*1': <Atom 'C.'>}
{'*1': <Atom 'C.'>}
{'*1': <Atom 'S.'>}
{'*1': <Atom 'C.'>}
{'*2': <Atom 'C'>, '*3': <Atom 'C.'>, '*4': <Atom 'H'>}
{'*2': <Atom 'C'>, '*3': <Atom 'C.'>, '*4': <Atom 'H'>}
{'*2': <Atom 'C'>, '*3': <Atom 'C.'>, '*4': <Atom 'H'>}
{'*1': <Atom 'C.'>}
{'*1': <Atom 'S.'>}
{'*1': <Atom 'C.'>

In [ ]:
# go through the new training reactions and make sure they use existing species dictionary definition
for entry in ark_kinetics_database.libraries[""].entries:
    for i in range(0, len(ark_kinetics_database.libraries[""].entries[entry].item.reactants)):
        old_reactant_name = ark_kinetics_database.libraries[""].entries[entry].item.reactants[i].label
        official_sp = get_species(ark_kinetics_database.libraries[""].entries[entry].item.reactants[i])
        if official_sp:
            ark_kinetics_database.libraries[""].entries[entry].item.reactants[i] = official_sp
            ark_kinetics_database.libraries[""].entries[entry].label = ark_kinetics_database.libraries[""].entries[entry].label.replace(
                old_reactant_name,
                official_sp.label
            )

    for i in range(0, len(ark_kinetics_database.libraries[""].entries[entry].item.products)):
        old_product_name = ark_kinetics_database.libraries[""].entries[entry].item.products[i].label
        official_sp = get_species(ark_kinetics_database.libraries[""].entries[entry].item.products[i])
        if official_sp:
            ark_kinetics_database.libraries[""].entries[entry].item.products[i] = official_sp
            ark_kinetics_database.libraries[""].entries[entry].label = ark_kinetics_database.libraries[""].entries[entry].label.replace(
                old_product_name,
                official_sp.label
            )


In [ ]:
# # need to convert the entry.item to an actual reaction instead of an entry
# for i, entry in enumerate(ark_kinetics_database.libraries[''].entries):
#     products = ark_kinetics_database.libraries[''].entries[entry].item.products
#     reactants = ark_kinetics_database.libraries[''].entries[entry].item.reactants
#     kinetics = ark_kinetics_database.libraries[''].entries[entry].item.kinetics
#     ark_kinetics_database.libraries[''].entries[entry].item = rmgpy.reaction.Reaction()
#     ark_kinetics_database.libraries[''].entries[entry].item.products = products
#     ark_kinetics_database.libraries[''].entries[entry].item.reactants = reactants
#     ark_kinetics_database.libraries[''].entries[entry].item.kinetics = kinetics

In [ ]:
for i, entry in enumerate(ark_kinetics_database.libraries[''].entries):
    print(type(ark_kinetics_database.libraries[''].entries[entry].item))
    break

In [ ]:
for entry in training_depo.entries:
    print(type(training_depo.entries[entry].item))
    break

In [ ]:
ark_kinetics_database.libraries[''].entries[213].item

In [ ]:
rmgpy.reaction.Reaction(ark_kinetics_database.libraries[''].entries[213].item)

In [ ]:
# Splice the new training reactions into the training depo
index_start = 100

for i, entry in enumerate(ark_kinetics_database.libraries[''].entries):
    
    if duplicate_exists(ark_kinetics_database.libraries[''].entries[entry]):
        continue
    if ark_kinetics_database.libraries[''].entries[entry].item.elementary_high_p:
        # manual fix to this issue: https://github.com/ReactionMechanismGenerator/RMG-Py/issues/2341
        ark_kinetics_database.libraries[''].entries[entry].item.elementary_high_p = False
    
    j = index_start + i
    while j in training_depo.entries.keys():
        j += 1
        continue
#     print(j, ark_kinetics_database.libraries[''].entries[entry])
    training_depo.entries[j] = ark_kinetics_database.libraries[''].entries[entry]
print(len(training_depo.entries), 'training reactions')

In [ ]:
for entry in training_depo.entries:
    print(training_depo.entries[entry].item.reactants[0].molecule[0].get_all_labeled_atoms())

In [ ]:
dir(training_depo.entries[15].item.reactants[0])

In [ ]:
dir(training_depo)

In [ ]:
for key in training_depo.entries:
    print(training_depo.entries[key], training_depo.entries[key].long_desc[0:4])

In [ ]:
# Save the results somewhere else
training_dir = os.path.join(rmgpy.settings['database.directory'], 'kinetics', 'families', family, 'training')
training_depo.save_dictionary(os.path.join(training_dir, 'dictionary.txt'))
training_depo.save(os.path.join(training_dir, 'reactions.py'))